In [ ]:
%pip install flask_cors

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
import numpy as np

from flask import Flask, request, jsonify
from flask_cors import CORS

import json
import logging

app = Flask(__name__)
CORS(app)
logging.basicConfig(level=logging.DEBUG)

valData = pd.read_csv('./valData.csv').drop(columns=['Unnamed: 0'])

def weightScore(chainScore, weight):
    chainScore['contribution_score'] = chainScore['contribution_score'] * weight[0] / sum(weight)
    chainScore['stability_score'] = chainScore['stability_score'] * weight[1] / sum(weight)
    chainScore['popularity_score'] = chainScore['popularity_score'] * weight[2] / sum(weight)
    chainScore['commission_score'] = chainScore['commission_score'] * weight[3] / sum(weight)
    chainScore['period_score'] = chainScore['period_score'] * weight[4] / sum(weight)
    chainScore['total_score'] = chainScore['contribution_score'] + chainScore['stability_score'] + chainScore['popularity_score'] + chainScore['commission_score'] + chainScore['period_score']
    return chainScore

@app.route('/')
def home():
    return jsonify(message="Hello from Flask")

@app.route('/getScore', methods=['POST'])
def getScore():
    try:
        data = request.get_json()

        chain = data['chain']
        weight = data['weight']

        chainData = valData[valData['chain']==chain]
        chainScore = chainData[['voter', 'contribution_score', 'stability_score', 'popularity_score', 'commission_score', 'period_score']]
        chainScore = weightScore(chainScore, weight).sort_values(by='total_score', ascending=False)

        json_str = chainScore.to_json(orient='records')

        return json_str, 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500

@app.route('/getValidator', methods=['POST'])
def getValidator():
    try:
        data = request.get_json()

        voter = data['voter']
        weight = data['weight']

        
        validator = valData[valData['voter']==voter]
        validator = weightScore(validator, weight).sort_values(by='total_score', ascending=False)

        asset_total = validator['asset_value'].sum()
        validator['asset_ratio'] = validator['asset_value'] / asset_total
        validator['weight_score'] = validator['total_score'] * validator['asset_ratio']

        validator = validator[['chain', 'contribution_score', 'stability_score', 'popularity_score', 'commission_score', 'period_score', 'total_score', 
                                'asset_value', 'asset_ratio', 'weight_score', 'rank', 'missblock', 'p_participation', 'p_passed', 'p_matchproposal', 'jailed_ratio']]

        json_str = validator.to_json(orient='records')

        return json_str, 200

    except Exception as e:
        print(f"An error occurred: {e}")
        return jsonify({"error": "Internal Server Error"}), 500


if __name__ == '__main__':
    app.run(port=5002)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit
